In [144]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
import duckdb
import pandas as pd
import xarray as xr
import numpy as np
from pyquadkey2 import quadkey

# Import jupysql Jupyter extension to create SQL cells
%load_ext sql

# configurations on jupysql to directly output data to Pandas and to simplify the output
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

In [ ]:
con = duckdb.connect("../data/qudkeyDB.duckdb") # this should create a DB File
con.install_extension("spatial")
con.load_extension("spatial")


CatalogException: Catalog Error: Table with name data_slice_male_quadkey_2 does not exist!
Did you mean "data_slice_male_long_lat"?
LINE 1: ...a_slice_male_long_lat AS SELECT * FROM data_slice_male_quadkey_2
                                                  ^

In [ ]:
# If the table does not already exists, it will be created
# feel free to rename columns
create_table_query = """
CREATE TABLE if not exists data_slice_male_long_lat (
    quadkey VARCHAR,
    raster_1 FLOAT,
    raster_2 FLOAT,
    raster_3 FLOAT,
    raster_4 FLOAT,
    raster_5 FLOAT,
    raster_6 FLOAT,
    raster_7 FLOAT,
    raster_8 FLOAT,
    raster_9 FLOAT,
    raster_10 FLOAT,
    raster_11 FLOAT,
    raster_12 FLOAT,
    raster_13 FLOAT,
    raster_14 FLOAT,
    raster_15 FLOAT,
    raster_16 FLOAT,
    raster_17 FLOAT,
    raster_18 FLOAT,
    raster_19 FLOAT,
    raster_20 FLOAT,
    raster_21 FLOAT,
    raster_22 FLOAT,
    raster_23 FLOAT,
    raster_24 FLOAT,
    raster_25 FLOAT,
    raster_26 FLOAT,
    raster_27 FLOAT,
    raster_28 FLOAT,
    raster_29 FLOAT,
    raster_30 FLOAT
);
"""

# Execute the query to create the table
con.execute(create_table_query)

In [ ]:
def insertIntoDB(df): # Insert dataframe into data_slice_male_long_lat
    con.sql(f"""
        INSERT INTO data_slice_male_long_lat
        SELECT * FROM df;
    """)


In [16]:
# deletes all data!!!!!!

#con.sql("""
#        truncate table data_slice_male_long_lat;
#    """)

In [ ]:
# loads the .nc file into an xarray 

def load_dataset(file_path, lat_slice, lon_slice):
    dataset = xr.open_dataset(file_path)
    # TODO: Fix out of memory problem by using dask https://tutorial.xarray.dev/intermediate/xarray_and_dask.html
    data_slice = dataset.sel(latitude=lat_slice, longitude=lon_slice)
    data_slice = data_slice[
        "Basic Demographic Characteristics, v4.10 (2010): Male, Density, 2.5 arc-minutes"
    ]
    return data_slice.compute()

In [ ]:
import pandas as pd
import numpy as np

# extraction of the data points from the xarray with their respective quadkeys

def extract_data_points_vectorized_quadkey(ds):
    data_points = []
    longs = ds.longitude.values
    lats = ds.latitude.values

    zoom_level = 14  # Adjust as needed for desired precision

    for x in range(len(lats)):  # Iterate over latitude indices
        for y in range(len(longs)):  # Iterate over longitude indices
            # Convert latitude and longitude to quadkey
            coord = (lats[x], longs[y])
            tile = quadkey.from_geo(coord, zoom_level)
            quadkey_str = tile.key
            
            # Extract raster values
            raster_values = ds.values[:, x, y]
            
            # Append a row with quadkey and raster values
            data_points.append([quadkey_str] + list(raster_values))

    # Create a DataFrame from the data points
    columns = ['quadkey'] + [f'raster_{i + 1}' for i in range(ds.shape[0])]
    df = pd.DataFrame(data_points, columns=columns)

    return df


In [ ]:
# function for loading chunks of 10°*10° into the DB
def load_male_datasetIntoDB():
    file_path_male = "../data/gpw_v4_basic_demographic_characteristics_rev11_mt_2010_dens_2pt5_min.nc"

    # Initialize an empty DataFrame to store results
    # combined_df = pd.DataFrame()

    # Process the dataset in 10-degree latitude chunks
    # for lat_start in range(85, -90, -10):  # Iterate from 90 to -90 in steps of -10
    for lat_start in range(85, -85, -10):  # Iterate from 90 to -90 in steps of -10
        lat_end = lat_start - 10
        lat_slice = slice(lat_start, lat_end)

        # for lon_start in range(-180, 180, 10):  # Iterate from 90 to -90 in steps of -10
        for lon_start in range(-180, 180, 10):  # Iterate from 90 to -90 in steps of -10
            lon_end = lon_start + 10
            lon_slice = slice(lon_start, lon_end)
            print("lat: " + str(lat_start) + " long: " + str(lon_start))
            print("lat: " + str(lat_end) + " long: " + str(lon_end))

            # Load and process the chunk
            data_slice_male = load_dataset(file_path_male, lat_slice, lon_slice)
            data_slice_male_quadkey = extract_data_points_vectorized_quadkey(data_slice_male)

            insertIntoDB(data_slice_male_quadkey)

In [ ]:
# n = zoomlevel must be parentlevel of the highest zoomlevel currently in the DB
def insertParentsForLevel(n, tableName):
    for group in range(0, 4): # run in chunks of 4, otherwise i ran out of memory; each group is defined by the first character of the quadkey (0-3)
        query = f"""
        INSERT INTO {tableName}
        SELECT
        SUBSTRING(quadkey, 1, {n} - 1) AS parent_quadkey,
        AVG(raster_1) * (COUNT(*) / 4) AS raster_1,
        AVG(raster_2) * (COUNT(*) / 4) AS raster_2,
        AVG(raster_3) * (COUNT(*) / 4) AS raster_3,
        AVG(raster_4) * (COUNT(*) / 4) AS raster_4,
        AVG(raster_5) * (COUNT(*) / 4) AS raster_5,
        AVG(raster_6) * (COUNT(*) / 4) AS raster_6,
        AVG(raster_7) * (COUNT(*) / 4) AS raster_7,
        AVG(raster_8) * (COUNT(*) / 4) AS raster_8,
        AVG(raster_9) * (COUNT(*) / 4) AS raster_9,
        AVG(raster_10) * (COUNT(*) / 4) AS raster_10,
        AVG(raster_11) * (COUNT(*) / 4) AS raster_11,
        AVG(raster_12) * (COUNT(*) / 4) AS raster_12,
        AVG(raster_13) * (COUNT(*) / 4) AS raster_13,
        AVG(raster_14) * (COUNT(*) / 4) AS raster_14,
        AVG(raster_15) * (COUNT(*) / 4) AS raster_15,
        AVG(raster_16) * (COUNT(*) / 4) AS raster_16,
        AVG(raster_17) * (COUNT(*) / 4) AS raster_17,
        AVG(raster_18) * (COUNT(*) / 4) AS raster_18,
        AVG(raster_19) * (COUNT(*) / 4) AS raster_19,
        AVG(raster_20) * (COUNT(*) / 4) AS raster_20,
        AVG(raster_21) * (COUNT(*) / 4) AS raster_21,
        AVG(raster_22) * (COUNT(*) / 4) AS raster_22,
        AVG(raster_23) * (COUNT(*) / 4) AS raster_23,
        AVG(raster_24) * (COUNT(*) / 4) AS raster_24,
        AVG(raster_25) * (COUNT(*) / 4) AS raster_25,
        AVG(raster_26) * (COUNT(*) / 4) AS raster_26,
        AVG(raster_27) * (COUNT(*) / 4) AS raster_27,
        AVG(raster_28) * (COUNT(*) / 4) AS raster_28,
        AVG(raster_29) * (COUNT(*) / 4) AS raster_29,
        AVG(raster_30) * (COUNT(*) / 4) AS raster_30
        FROM {tableName}
        WHERE LENGTH(quadkey) > 1  -- Exclude keys that are already parents
        and SUBSTRING(quadkey,1,1) = '{group}'
        GROUP BY parent_quadkey
        """
        # Execute the query
        con.execute(query).fetchall()

In [ ]:
# for each level all parents are insertet into the db; start with 14 when the xarray has been freshly loaded
# dont run if the db is alreay filled
def loadAllAggregationDataIntoDB():
    n = 14
    for n in range(n, 0, -1):
        print("level: " + str(n))
        insertParentsForLevel(n, "data_slice_male_long_lat")


In [ ]:
# delete higher levels if you fucked up
# con.sql("""
#         delete  FROM data_slice_male_long_lat
#         where LENGTH(quadkey) < 13
#     """)

In [ ]:
# Loads the complete male Dataset at level 14 into the DB
# only run if the DB has no data inside

load_male_datasetIntoDB() # loads level 14 from source; took 10min for me
loadAllAggregationDataIntoDB() # loads aggregation for all levels; took 5min for me



In [ ]:
# validate data
con.sql("""
        SELECT * FROM data_slice_male_long_lat
    """)



┌────────────────┬──────────┬──────────┬──────────┬──────────┬──────────┬──────────┬──────────┬──────────┬──────────┬───────────┬───────────┬───────────┬───────────┬───────────┬───────────┬───────────┬───────────┬───────────┬───────────┬───────────┬───────────┬───────────┬───────────┬───────────┬───────────┬───────────┬───────────┬───────────┬───────────┬───────────┐
│    quadkey     │ raster_1 │ raster_2 │ raster_3 │ raster_4 │ raster_5 │ raster_6 │ raster_7 │ raster_8 │ raster_9 │ raster_10 │ raster_11 │ raster_12 │ raster_13 │ raster_14 │ raster_15 │ raster_16 │ raster_17 │ raster_18 │ raster_19 │ raster_20 │ raster_21 │ raster_22 │ raster_23 │ raster_24 │ raster_25 │ raster_26 │ raster_27 │ raster_28 │ raster_29 │ raster_30 │
│    varchar     │  float   │  float   │  float   │  float   │  float   │  float   │  float   │  float   │  float   │   float   │   float   │   float   │   float   │   float   │   float   │   float   │   float   │   float   │   float   │   float   │   float   

In [ ]:
# close connection
# otherwise the fastapi might not work
con.close()